In [8]:
# IMPORT
import numpy as np
from joblib import Parallel, delayed
from ipynb.fs.full.utility_functions import gen_data_dict, get_data, error_count, write_to_csv, opart

In [10]:
def try_lambda(i):
    lda = ldas[i]
    chpnt = opart(lda, sequence)
    err_1 = error_count(chpnt, neg_start_1, neg_end_1, pos_start_1, pos_end_1)[:2]
    err_2 = error_count(chpnt, neg_start_2, neg_end_2, pos_start_2, pos_end_2)[:2]
    return err_1, err_2
    
if __name__ == "__main__":
    # get sequences and labels
    seqs   = gen_data_dict('sequence_label_data/genome/signals.gz')
    labels = gen_data_dict('sequence_label_data/genome/labels.gz')

    # lambda candidates
    ldas = [np.e**element for element in [x/4 for x in range(-28, 29)]]

    for i in range(len(seqs)):
        # generate data
        sequence, neg_start_1, neg_end_1, pos_start_1, pos_end_1, neg_start_2, neg_end_2, pos_start_2, pos_end_2 = get_data(i, seqs=seqs, labels=labels)

        # run each lambda and record it into csv file
        row  = [seqs[i][0]]
        errs = Parallel(n_jobs=16)(delayed(try_lambda)(i) for i in range(0, len(ldas)))
        row += errs

        header = ['sequenceID'] + [str(num) for num in [x/4 for x in range(-28, 29)]]
        write_to_csv('1.genome_learning_data/errors_base_e.csv', header, row)